# Das Grundprinzip von Gruppieren
Der Prozess des Gruppieren ("group by") bezieht sich auf ein oder mehrere Handlung:
* **Splitting** Also das Zerlegen des Datensatzes in bestimmte Gruppen
* **Applying** Anwenden einer Funktion auf jede dieser Gruppen
* **Combining** Zusammenfassen der Daten in eine neue Datenstruktur

Das **Splitting** ist der zentrale Schritt und wird oft als erstes ausgeführt. Meisten sollen die Datensätze in den Gruppen aber noch weiterverarbeitet werden. Daher stehenim Schritt **Applying** unterem anderem die folgenden Funktionen zur Verfügung:

* **Aggregation:** Kummulieren der Daten mit Statistischen Methoden, wie zum Beispiel:
    * Summen und Mittelwerte
    * Zählen der Werte
    
    
* **Transformation:**
    * Standardisierung der Daten innerhalb einer Gruppe
    * Aufüllen von fehlenden Daten innerhalb einrer Gruppe mit aus der Gruppe abgeleitetend Werte

* **Filtration:**
    * Gruppenbezogenes Filtern, z.B. das Aussortieren von Ausreißern innerhalb einer Gruppe
    

* Oder aber die Kombination verschiedener Schritte



# Arbeiten mit *.groupby()*
## Datageneration

In [42]:
import pandas as pd
import numpy as np

In [43]:
# Generieren von Übungsdaten
df_students = pd.DataFrame({'student_id':list(range(100,107)), 
                            'subject_id':[1,2,3,1,2,3,1],
                            'subject': ['Math', 'Geo', 'Business', 'Math', 'Geo', 'Business' ,'Math'],
                            'age':[18,22,31,22,26,21,20],
                           'left_hand':[False,True,False,True,False,False,True]})
df_students

,student_id,subject_id,subject,age,left_hand
0,100,1,Math,18,False
1,101,2,Geo,22,True
2,102,3,Business,31,False
3,103,1,Math,22,True
4,104,2,Geo,26,False
5,105,3,Business,21,False
6,106,1,Math,20,True


## Zählen von Datensätzen

In [44]:
# Mit groupby.count() können wir uns anzeigen lassen, wie viel Datensätze jede Gruppe besitzt 
df_students.groupby('subject').count()

,student_id,subject_id,age,left_hand
subject,,,,
Business,2,2,2,2
Geo,2,2,2,2
Math,3,3,3,3


In [45]:
# Alternativ können wir auch nach der Größe fragen
df_students.groupby('subject').size()

subject
Business    2
Geo         2
Math        3
dtype: int64

**Unterschied zwischen ``.count()`` und ``.size()``**

In [61]:
# Unterschied zwischen .count() und .size()
df_students_nan = pd.DataFrame({'student_id':list(range(100,107)), 
                            'subject_id':[1,2,3,1,2,np.NaN,1],
                            'subject': ['Math', 'Geo', 'Business', 'Math', 'Geo', 'Business' ,'Math'],
                            'age':[np.NaN,22,31,22,26,21,20],
                           'left_hand':[False,True,False,True,False,False,True]})
df_students_nan

,student_id,subject_id,subject,age,left_hand
0,100,1.0,Math,NaN,False
1,101,2.0,Geo,22.0,True
2,102,3.0,Business,31.0,False
3,103,1.0,Math,22.0,True
4,104,2.0,Geo,26.0,False
5,105,NaN,Business,21.0,False
6,106,1.0,Math,20.0,True


In [62]:
# count wird durch NaN Werte beeinflusst
df_students_nan.groupby('subject').count()

,student_id,subject_id,age,left_hand
subject,,,,
Business,2,1,2,2
Geo,2,2,2,2
Math,3,3,2,3


In [64]:
# Size verhindert diesen Effekt
df_students_nan.groupby('subject').size()

subject
Business    2
Geo         2
Math        3
dtype: int64

## Wie der Index entsteht

In [48]:
# Da wir nach Subject gruppieren, werden die einzigeartigen Werte der Spalte "subject" der neue Index. 
# Den enstehenden Index können wir auch generieren wenn wir uns die einzigartigen Werte der Spalte "subject" anschauen
df_students['subject'].unique()

array(['Math', 'Geo', 'Business'], dtype=object)

## Aggregation von Daten

Wichtigste Funktionen für die Aggregation:
* .sum() - *Summe*
* .mean() - *Mittelwert*
* .min() - *Minimalwert*
* .max() - *Maximalwert*

In [49]:
# Bilden der Summe nach der "subject_id", text wird ignoriert
df_students.groupby('subject_id').sum()

,student_id,age,left_hand
subject_id,,,
1,309,60,2.0
2,205,48,1.0
3,207,52,0.0


In [50]:
# Durchschnittsalter berechnen
df_students[['subject','age']].groupby('subject').mean()

,age
subject,
Business,26
Geo,24
Math,20


## Auszug aus dem Dataframe betrachten
Mit ``.first()`` und ``.last()`` bekommen wir die einen neuen Dataframe zurück in dem nur der erste oder der letzte Datensatz einer jeden Gruppe enthalten ist. Dies kann besonders nützlich sein, wenn die Datensätze in einer bestimmen Reihenfolge im Datensatz auftauchen.

In [51]:
df_students.groupby('subject').first()

,student_id,subject_id,age,left_hand
subject,,,,
Business,102,3,31,False
Geo,101,2,22,True
Math,100,1,18,False


In [52]:
df_students.groupby('subject').first()

,student_id,subject_id,age,left_hand
subject,,,,
Business,102,3,31,False
Geo,101,2,22,True
Math,100,1,18,False


**Auch ``.head()`` und ``.tail()`` lassen sich hier wie gewohnt verwenden**

In [53]:
df_students.groupby('subject').head(2)

,student_id,subject_id,subject,age,left_hand
0,100,1,Math,18,False
1,101,2,Geo,22,True
2,102,3,Business,31,False
3,103,1,Math,22,True
4,104,2,Geo,26,False
5,105,3,Business,21,False


## Mehrere Gruppierungsebenen
Oft bleibt es nicht bei einer Gruppierungsebene. Wie können beliebig viele Gruppierungsebenen hinzufügen. Die Reihenfolge hat zunächst keinen Erkennbaren Einfluss auf das Endergebnis, außer, dass sich die Sortierung der Daten ändert.

In [54]:
# Größeres Datenset einlesen
df = pd.read_csv('../src/bigdata/120-years-of-olympic-history-athletes-and-results/athlete_events.csv')
df.head(5)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [55]:
# Uns interessieren die Medallien und zwar nur die Gewinne
df['Medal'].unique()

array([nan, 'Gold', 'Bronze', 'Silver'], dtype=object)

**Umgang mit fehlenden oder fehlerhaften Werten, mit ``.dropna()`` und ``.fillna()``**

In [56]:
# Eine Möglichkeit ist die Daten zu verwerfen
df_drop = df.dropna(subset=['Medal'])
print(len(df))
print(len(df_drop))

271116
39783


In [57]:
# Bessser jedoch ist, wenn wir den fehlenden Daten eine Bedeutung und einen Wert zukommen lassen
df['Medal'] = df['Medal'].fillna('Teilnahme')
df.head(5)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Teilnahme
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,Teilnahme
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,Teilnahme
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,Teilnahme


In [58]:
# Teilnehmer ist einzigartig zugeordnet über die ID. Sie bildet das oberste Level.
df.groupby(['ID', 'Name', 'Team', 'Medal']).size().head(5)

ID  Name                      Team            Medal    
1   A Dijiang                 China           Teilnahme    1
2   A Lamusi                  China           Teilnahme    1
3   Gunnar Nielsen Aaby       Denmark         Teilnahme    1
4   Edgar Lindenau Aabye      Denmark/Sweden  Gold         1
5   Christine Jacoba Aaftink  Netherlands     Teilnahme    6
dtype: int64

In [65]:
df.groupby(['Medal', 'ID', 'Name', 'Team']).size().head(5)

Medal   ID  Name                      Team       
Bronze  15  Arvo Ossian Aaltonen      Finland        1
        16  Juhamatti Tapio Aaltonen  Finland        1
        17  Paavo Johannes Aaltonen   Finland        1
        20  Kjetil Andr Aamodt        Norway         1
        29  Willemien Aardenburg      Netherlands    1
dtype: int64

**Gruppierung zurücksetzen und in "normalen" DataFrame zurückverwandeln mit ``.reset_index()`` und umbennen von Spalten mit ``.rename()``**  

In [59]:
# Zurück wandeln eines "grouped" DataFrames mit .reset_index() 
df = df.groupby(['ID', 'Name', 'Team', 'Medal']).size().reset_index()
df.rename(columns={0:'Anzahl'}).head(5)

,ID,Name,Team,Medal,Anzahl
0,1,A Dijiang,China,Teilnahme,1
1,2,A Lamusi,China,Teilnahme,1
2,3,Gunnar Nielsen Aaby,Denmark,Teilnahme,1
3,4,Edgar Lindenau Aabye,Denmark/Sweden,Gold,1
4,5,Christine Jacoba Aaftink,Netherlands,Teilnahme,6


# Tipps and Tricks

###  Referenz aller Funktionen des groupby-Objekt
* https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats

### Vollständige Tutorial zu Gruppieren aus der offiziellen Pandas Dokumentation
* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
* https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-grouping
    
